# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

One Hot Encoding the "Geography" column

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

### Splitting the dataset into the Training set and Test set

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [25]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN model

In [26]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [27]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5953 - accuracy: 0.7160
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4941 - accuracy: 0.7956
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4648 - accuracy: 0.7976
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4470 - accuracy: 0.8008
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4367 - accuracy: 0.8075
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4296 - accuracy: 0.8129
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4230 - accuracy: 0.8151
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4160 - accuracy: 0.8183
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4084 - accuracy: 0.8241
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3995 - accura

## Making the predictions and evaluating the model

### Predicting the result of single observations

Using the model to predict if the customers with the following informations will leave the bank: 

1. Geography: France, Score: 600, Sex: M/F, Age: 40yrs, Tenure: 3yrs, Balance: $60K, Num of Products: 2, Credit card: Yes, Active: Yes Estimated Salary: $50K
2. Geography: Spain, Score: 350, Sex: M/F, Age: 42yrs, Tenure: 8yrs, Balance: $160K, Num of Products: 3, Credit card: Yes, Active: No Estimated Salary: $114K
3. Geography: Germany, Score: 674, Sex: M/F, Age: 54yrs, Tenure: 4yrs, Balance: $0, Num of Products: 1, Credit card: Yes, Active: No Estimated Salary: $91K
4. Geography: France, Score: 900, Sex: M/F, Age: 42yrs, Tenure: 8yrs, Balance: $100K, Num of Products: 1, Credit card: Yes, Active: Yes Estimated Salary: $72K

In [46]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
print(ann.predict(sc.transform([[1, 0, 0, 600, 0, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 16ms/step
[[False]]
1/1 [==============================] - 0s 16ms/step
[[False]]


In [47]:
print(ann.predict(sc.transform([[0, 0, 1, 350, 1, 42, 8, 160000, 3, 1, 0, 114000]])) > 0.5)
print(ann.predict(sc.transform([[0, 0, 1, 350, 0, 42, 8, 160000, 3, 1, 0, 114000]])) > 0.5)

1/1 [==============================] - 0s 13ms/step
[[ True]]
1/1 [==============================] - 0s 24ms/step
[[ True]]


In [83]:
print(ann.predict(sc.transform([[0, 1, 0, 674, 1, 54, 4, 0, 1, 1, 0, 91000]])) > 0.5)
print(ann.predict(sc.transform([[0, 1, 0, 674, 0, 42, 4, 0, 1, 1, 0, 91000]])) > 0.5)

1/1 [==============================] - 0s 16ms/step
[[ True]]
1/1 [==============================] - 0s 16ms/step
[[ True]]


In [91]:
print(ann.predict(sc.transform([[1, 0, 0, 900, 1, 25, 6, 100000, 1, 1, 1, 72000]])) > 0.5)
print(ann.predict(sc.transform([[1, 0, 0, 900, 0, 25, 6, 100000, 1, 1, 1, 72000]])) > 0.5)

1/1 [==============================] - 0s 16ms/step
[[False]]
1/1 [==============================] - 0s 16ms/step
[[False]]


From the above observations, we can may make the following conclusions:
1. Atleast gender has little to no effect on whether the person leaves the bank or not and hence it proves to be an almost useless feature for our analysis
2. It appears, that a person who is an inactive member, or has minimal balance in the bank is quite likely to leave
3. Younger members are more likely to stay
4. The number of products a person has does not affect their exit from the bank to much extent

More such conclusions can be made on making more such observations. The above observations may not be a 100% true, becausse we have made these conclusions using a very small number of predictions, but the process remains the same. 

### Predicting the Test set results

In [92]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
y_pred.shape

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


(2000, 1)

### Making the Confusion Matrix

In [93]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1524   71]
 [ 196  209]]


0.8665